## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-31-04-05-36 +0000,nyt,Here’s the latest.,https://www.nytimes.com/2026/01/30/us/heres-th...
1,2026-01-31-04-03-13 +0000,eltecolote,"Students, teachers walk out as San Francisco j...",https://eltecolote.org/content/en/ice-walkout-...
2,2026-01-31-04-00-56 +0000,latimes,"Planned Parenthood, reproductive healthcare co...",https://www.latimes.com/california/story/2026-...
3,2026-01-31-04-00-00 +0000,wsj,Why China Cares About Being Cool All of a Sudden,https://www.wsj.com/world/china/labubu-youtube...
4,2026-01-31-04-00-00 +0000,wsj,High-Profile Arrests Spark Exodus of Scam Work...,https://www.wsj.com/world/asia/high-profile-ar...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2509/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
171,trump,73
60,epstein,24
12,ice,24
347,fed,23
356,warsh,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
381,2026-01-30-04-50-00 +0000,wsj,President Trump’s advisers have been told that...,https://www.wsj.com/economy/central-banking/tr...,184
330,2026-01-30-11-18-12 +0000,cbc,Trump names former Fed official turned critic ...,https://www.cbc.ca/news/world/trump-federal-re...,181
222,2026-01-30-17-24-50 +0000,nypost,Gold and silver plunge after news Trump will n...,https://nypost.com/2026/01/30/business/gold-si...,177
246,2026-01-30-16-20-00 +0000,wsj,"Kevin Warsh, a central-bank insider-turned-cri...",https://www.wsj.com/economy/central-banking/th...,160
147,2026-01-30-21-08-13 +0000,nyt,"Kevin Warsh, Trump’s Choice for Fed Chair Coul...",https://www.nytimes.com/2026/01/30/business/tr...,158


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
381,184,2026-01-30-04-50-00 +0000,wsj,President Trump’s advisers have been told that...,https://www.wsj.com/economy/central-banking/tr...
52,89,2026-01-31-01-25-43 +0000,bbc,Former CNN host Don Lemon charged over anti-IC...,https://www.bbc.com/news/articles/c24gm3dz36po...
29,80,2026-01-31-02-39-00 +0000,wsj,With hours to go before a partial government s...,https://www.wsj.com/politics/policy/senate-set...
179,78,2026-01-30-19-56-00 +0000,wsj,Kevin Warsh’s nomination to lead the Federal R...,https://www.wsj.com/economy/central-banking/wh...
216,63,2026-01-30-17-52-43 +0000,nypost,New Ghislaine Maxwell mugshot revealed in late...,https://nypost.com/2026/01/30/us-news/new-ghis...
38,54,2026-01-31-02-09-07 +0000,nypost,Mamdani still will only force homeless NYers o...,https://nypost.com/2026/01/30/us-news/mamdani-...
67,41,2026-01-31-00-28-00 +0000,wsj,In her Amazon-backed documentary film “Melania...,https://www.wsj.com/arts-culture/film/melania-...
95,40,2026-01-30-23-12-35 +0000,nypost,Disney CEO Bob Iger plans to step down before ...,https://nypost.com/2026/01/30/business/disney-...
1,38,2026-01-31-04-03-13 +0000,eltecolote,"Students, teachers walk out as San Francisco j...",https://eltecolote.org/content/en/ice-walkout-...
32,37,2026-01-31-02-31-23 +0000,latimes,Ex-Google engineer convicted of stealing AI tr...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
